In [1]:
!pip install pandas
!pip install numpy
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 23.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 21.3 MB/s eta 0:00:00a 0:00:01


In [2]:
import sys
!{sys.executable} -m pip install pandas numpy scikit-learn

In [3]:
import pandas as pd
import numpy as np
import re 
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

class RecommendationModel():
    def __init__(self):
        # Data Preparation
        spotify_df = pd.read_csv('data.csv')
        data_w_genre = pd.read_csv('data_w_genres.csv')
        data_w_genre['genres_upd'] = data_w_genre['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])
        spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
        spotify_df['artists_upd_v2'] = spotify_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
        spotify_df['artists_upd'] = np.where(spotify_df['artists_upd_v1'].apply(lambda x: not x), spotify_df['artists_upd_v2'], spotify_df['artists_upd_v1'])
        spotify_df['artists_song'] = spotify_df.apply(lambda row: row['artists_upd'][0]+row['name'],axis=1)
        spotify_df.sort_values(['artists_song','release_date'], ascending=False, inplace=True)
        spotify_df.drop_duplicates('artists_song', inplace=True)
        artists_exploded = spotify_df[['artists_upd','id']].explode('artists_upd')
        artists_exploded_enriched = artists_exploded.merge(
            data_w_genre, how='left', left_on='artists_upd', right_on='artists')
        artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()]
        artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id')['genres_upd'].apply(list).reset_index()
        artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(
            lambda x: list(set(list(itertools.chain.from_iterable(x)))))
        spotify_df = spotify_df.merge(
            artists_genres_consolidated[['id','consolidates_genre_lists']], on='id', how='left')
        spotify_df['year'] = spotify_df['release_date'].apply(lambda x: x.split('-')[0])
        float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values
        spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))
        spotify_df['genre'] = spotify_df['consolidates_genre_lists'].apply(
            lambda d: d if isinstance(d, list) else [])
        
        # Feature Engineering
        def ohe_prep(df, column, new_name): 
            tf_df = pd.get_dummies(df[column])
            feature_names = tf_df.columns
            tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
            tf_df.reset_index(drop=True, inplace=True)    
            return tf_df
        
        # def create_feature_set(df, float_cols):
        #     tfidf = TfidfVectorizer()
        #     tfidf_matrix = tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
        #     genre_df = pd.DataFrame(tfidf_matrix.toarray())
        #     genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
        #     genre_df.reset_index(drop=True, inplace=True)
        #     year_ohe = ohe_prep(df, 'year','year') * 0.5
        #     popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15
        #     floats = df[float_cols].reset_index(drop=True)
        #     scaler = MinMaxScaler()
        #     floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns=floats.columns) * 0.2
        #     final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis=1)
        #     final['id'] = df['id'].values
        #     return final
        def create_feature_set(df, float_cols):
            tfidf = TfidfVectorizer()
            # Ensure 'consolidates_genre_lists' contains lists
            df['consolidates_genre_lists'] = df['consolidates_genre_lists'].apply(lambda x: x if isinstance(x, list) else [])
            # Apply TF-IDF Vectorizer
            tfidf_matrix = tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
            genre_df = pd.DataFrame(tfidf_matrix.toarray())
            genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
            genre_df.reset_index(drop=True, inplace=True)
            
            # One-Hot Encoding for 'year' and 'popularity_red'
            year_ohe = ohe_prep(df, 'year', 'year') * 0.5
            popularity_ohe = ohe_prep(df, 'popularity_red', 'pop') * 0.15
            
            # Scale float columns
            floats = df[float_cols].reset_index(drop=True)
            scaler = MinMaxScaler()
            floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns=floats.columns) * 0.2
            
            # Concatenate all features
            final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis=1)
            final['id'] = df['id'].values
            return final


        complete_feature_set = create_feature_set(spotify_df, float_cols=float_cols)
        self.spotify_df = spotify_df
        self.complete_feature_set = complete_feature_set

    def create_necessary_outputs(self, playlist_ids):
        playlist = pd.DataFrame()
        for ix, song_id in enumerate(playlist_ids):
            if song_id in self.spotify_df['id'].values:
                song_data = self.spotify_df[self.spotify_df['id'] == song_id].iloc[0]
                playlist.loc[ix, 'artist'] = song_data['artists_upd'][0]
                playlist.loc[ix, 'name'] = song_data['name']
                playlist.loc[ix, 'id'] = song_id
                playlist.loc[ix, 'date_added'] = pd.to_datetime('today') - pd.Timedelta(days=ix)
        playlist['date_added'] = pd.to_datetime(playlist['date_added'])
        playlist = playlist[playlist['id'].isin(self.spotify_df['id'].values)].sort_values('date_added', ascending=False)
        return playlist

    def generate_playlist_feature(self, complete_feature_set, playlist_df, weight_factor=1.09):
        complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
        complete_feature_set_playlist = complete_feature_set_playlist.merge(
            playlist_df[['id','date_added']], on='id', how='inner')
        complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]
        playlist_feature_set = complete_feature_set_playlist.sort_values('date_added', ascending=False)
        most_recent_date = playlist_feature_set.iloc[0]['date_added']
        for ix, row in playlist_feature_set.iterrows():
            playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row['date_added'].to_pydatetime()).days / 30)
        playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(
            lambda x: weight_factor ** (-x))
        playlist_feature_set_weighted = playlist_feature_set.copy()
        playlist_feature_set_weighted.update(
            playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight, 0))
        playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
        return playlist_feature_set_weighted_final.sum(axis=0), complete_feature_set_nonplaylist

    def generate_playlist_recos(self, features, nonplaylist_features, top_k=10):
        non_playlist_df = self.spotify_df[self.spotify_df['id'].isin(nonplaylist_features['id'].values)]
        non_playlist_df = non_playlist_df.reset_index(drop=True)
        nonplaylist_features = nonplaylist_features.reset_index(drop=True)
        non_playlist_df['sim'] = cosine_similarity(
            nonplaylist_features.drop('id', axis=1).values, features.values.reshape(1, -1))[:,0]
        non_playlist_df_top_k = non_playlist_df.sort_values('sim', ascending=False).head(top_k)
        return non_playlist_df_top_k


In [4]:
##Example 1

# Instantiate the model
model = RecommendationModel()

# Provide a list of song IDs from your dataset
playlist_ids = ['6wn61Fzx9XMxQmieLpoIhW', '1D4PL9B8gOg78jiHg3FvBb', '5P4wWhUYWM0IaVYLuZxdar']

# Create playlist dataframe
playlist_df = model.create_necessary_outputs(playlist_ids)

# Generate playlist feature vector and non-playlist features
features, nonplaylist_features = model.generate_playlist_feature(model.complete_feature_set, playlist_df)

# Generate recommendations with top_k=1 to get only the top recommendation
recommendations = model.generate_playlist_recos(features, nonplaylist_features, top_k=1)

# View the top recommendation
print(recommendations[['id', 'artists_upd', 'name', 'sim']])


                           id     artists_upd     name      sim
27764  4t0OI7XrODjSkAu3bTPmWj  [Taylor Swift]  Fifteen  0.99909


In [5]:
##Example 2

# Instantiate the model
model = RecommendationModel()

# Provide a list of song IDs from your dataset
playlist_ids = ['7BqBn9nzAq8spo5e7cZ0dJ', '3RH9idbxUAMcUldet2ormp', '4HlFJV71xXKIGcU3kRyttv']

# Create playlist dataframe
playlist_df = model.create_necessary_outputs(playlist_ids)

# Generate playlist feature vector and non-playlist features
features, nonplaylist_features = model.generate_playlist_feature(model.complete_feature_set, playlist_df)

# Generate recommendations with top_k=1 to get only the top recommendation
recommendations = model.generate_playlist_recos(features, nonplaylist_features, top_k=1)

# View the top recommendation
print(recommendations[['id', 'artists_upd', 'name', 'sim']])


KeyboardInterrupt: 

In [ ]:
##Example 3

# Instantiate the model
model = RecommendationModel()

# Provide a list of song IDs from your dataset
playlist_ids = ['6Hj9jySrnFppAI0sEMCZpJ', '0nbXyq5TXYPCO7pr3N8S4I', '3eekarcy7kvN4yt5ZFzltW']

# Create playlist dataframe
playlist_df = model.create_necessary_outputs(playlist_ids)

# Generate playlist feature vector and non-playlist features
features, nonplaylist_features = model.generate_playlist_feature(model.complete_feature_set, playlist_df)

# Generate recommendations with top_k=1 to get only the top recommendation
recommendations = model.generate_playlist_recos(features, nonplaylist_features, top_k=1)

# View the top recommendation
print(recommendations[['id', 'artists_upd', 'name', 'sim']])

                           id              artists_upd  \
74321  2WsuSYJNXGKXVYkHPnq2yp  [Lil Tecca, Juice WRLD]   

                                   name       sim  
74321  Ransom (with Juice WRLD) - Remix  0.900897  


In [ ]:
# Print all song IDs
spotify_df = pd.read_csv('data.csv')
print(spotify_df[['id', 'name', 'artists']])

                            id  \
0       4BJqT0PrAfrxzMOxytFOIz   
1       7xPhfUan2yNtyFG0cUWkt8   
2       1o6I8BglA6ylDMrIELygv1   
3       3ftBPsC5vPBKxYSee08FDH   
4       4d6HGyGT8e121BsdKmw9v6   
...                        ...   
170648  0KkIkfsLEJbrcIhYsCL7L5   
170649  0OStKKAuXlxA0fMH54Qs6E   
170650  4BZXVFYCb76Q0Klojq4piV   
170651  5SiZJoLXp3WOl3J4C8IK0d   
170652  7HmnJHfs0BkFzX4x8j0hkl   

                                                     name  \
0       Piano Concerto No. 3 in D Minor, Op. 30: III. ...   
1                                 Clancy Lowered the Boom   
2                                               Gati Bali   
3                                               Danny Boy   
4                             When Irish Eyes Are Smiling   
...                                                   ...   
170648                                              China   
170649                        Halloweenie III: Seven Days   
170650                                    